<a href="https://colab.research.google.com/github/Francisss3/AAI614_Francis-/blob/main/Dash_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
%cd /content/drive/MyDrive/dash_drug_discovery/dash-drug-discovery


/content/drive/MyDrive/dash_drug_discovery/dash-drug-discovery


In [10]:
!ls


app.py	data		    helpers.py	__pycache__  requirements.txt
assets	drug_discovery.png  Procfile	README.md


In [11]:
!pip install dash jupyter_dash pandas gunicorn plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [12]:
from jupyter_dash import JupyterDash  # Use JupyterDash for Colab
import pandas as pd
import pathlib
import dash_html_components as html
import dash_core_components as dcc

from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from helpers import make_dash_table, create_plot  # Ensure helpers.py is accessible

# Initialize JupyterDash app
app = JupyterDash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)

DATA_PATH = pathlib.Path("./data").resolve()  # Adjust path for Colab

# Read from datasheet
df = pd.read_csv(DATA_PATH.joinpath("small_molecule_drugbank.csv")).drop(
    ["Unnamed: 0"], axis=1
)

STARTING_DRUG = "Levobupivacaine"
DRUG_DESCRIPTION = df.loc[df["NAME"] == STARTING_DRUG]["DESC"].iloc[0]
DRUG_IMG = df.loc[df["NAME"] == STARTING_DRUG]["IMG_URL"].iloc[0]
FIGURE = create_plot(
    x=df["PKA"],
    y=df["LOGP"],
    z=df["SOL"],
    size=df["MW"],
    color=df["MW"],
    name=df["NAME"],
)

app.layout = html.Div(
    [
        html.Div(
            [html.Img(src=app.get_asset_url("dash-logo.png"))], className="app__banner"
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.Div(
                            [
                                html.H3(
                                    "dash for drug discovery",
                                    className="uppercase title",
                                ),
                                html.Span("Hover ", className="uppercase bold"),
                                html.Span(
                                    "over a drug in the graph to see its structure."
                                ),
                                html.Br(),
                                html.Span("Select ", className="uppercase bold"),
                                html.Span(
                                    "a drug in the dropdown to add it to the drug candidates at the bottom."
                                ),
                            ]
                        )
                    ],
                    className="app__header",
                ),
                html.Div(
                    [
                        dcc.Dropdown(
                            id="chem_dropdown",
                            multi=True,
                            value=[STARTING_DRUG],
                            options=[{"label": i, "value": i} for i in df["NAME"]],
                        )
                    ],
                    className="app__dropdown",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                dcc.RadioItems(
                                    id="charts_radio",
                                    options=[
                                        {"label": "3D Scatter", "value": "scatter3d"},
                                        {"label": "2D Scatter", "value": "scatter"},
                                        {
                                            "label": "2D Histogram",
                                            "value": "histogram2d",
                                        },
                                    ],
                                    labelClassName="radio__labels",
                                    inputClassName="radio__input",
                                    value="scatter3d",
                                    className="radio__group",
                                ),
                                dcc.Graph(
                                    id="clickable-graph",
                                    hoverData={"points": [{"pointNumber": 0}]},
                                    figure=FIGURE,
                                ),
                            ],
                            className="two-thirds column",
                        ),
                        html.Div(
                            [
                                html.Div(
                                    [
                                        html.Img(
                                            id="chem_img",
                                            src=DRUG_IMG,
                                            className="chem__img",
                                        )
                                    ],
                                    className="chem__img__container",
                                ),
                                html.Div(
                                    [
                                        html.A(
                                            STARTING_DRUG,
                                            id="chem_name",
                                            href="https://www.drugbank.ca/drugs/DB01002",
                                            target="_blank",
                                        ),
                                        html.P(DRUG_DESCRIPTION, id="chem_desc"),
                                    ],
                                    className="chem__desc__container",
                                ),
                            ],
                            className="one-third column",
                        ),
                    ],
                    className="container card app__content bg-white",
                ),
                html.Div(
                    [
                        html.Table(
                            make_dash_table([STARTING_DRUG], df),
                            id="table-element",
                            className="table__container",
                        )
                    ],
                    className="container bg-white p-0",
                ),
            ],
            className="app__container",
        ),
    ]
)


def df_row_from_hover(hoverData):
    """ Returns row for hover point as a Pandas Series. """

    try:
        point_number = hoverData["points"][0]["pointNumber"]
        molecule_name = str(FIGURE["data"][0]["text"][point_number]).strip()
        return df.loc[df["NAME"] == molecule_name]
    except KeyError as error:
        print(error)
        return pd.Series()


@app.callback(
    Output("clickable-graph", "figure"),
    [Input("chem_dropdown", "value"), Input("charts_radio", "value")],
)
def highlight_molecule(chem_dropdown_values, plot_type):
    """
    Selected chemical dropdown values handler.

    :params chem_dropdown_values: selected dropdown values
    :params plot_type: selected plot graph
    """

    return create_plot(
        x=df["PKA"],
        y=df["LOGP"],
        z=df["SOL"],
        size=df["MW"],
        color=df["MW"],
        name=df["NAME"],
        markers=chem_dropdown_values,
        plot_type=plot_type,
    )


@app.callback(Output("table-element", "children"), [Input("chem_dropdown", "value")])
def update_table(chem_dropdown_value):
    """
    Update the table rows.

    :params chem_dropdown_values: selected dropdown values
    """

    return make_dash_table(chem_dropdown_value, df)


@app.callback(
    [
        Output("chem_name", "children"),
        Output("chem_name", "href"),
        Output("chem_img", "src"),
        Output("chem_desc", "children"),
    ],
    [Input("clickable-graph", "hoverData")],
)
def chem_info_on_hover(hoverData):
    """
    Display chemical information on graph hover.
    Update the image, link, description.

    :params hoverData: data on graph hover
    """

    if hoverData is None:
        raise PreventUpdate

    try:
        row = df_row_from_hover(hoverData)
        if row.empty:
            raise Exception
        return (
            row["NAME"].iloc[0],
            row["PAGE"].iloc[0],
            row["IMG_URL"].iloc[0],
            row["DESC"].iloc[0],
        )

    except Exception as error:
        print(error)
        raise PreventUpdate


# Run the app inline in Colab
app.run_server(mode='inline', debug=True)


<ipython-input-12-b587db580190>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-12-b587db580190>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/usr/local/lib/python3.10/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>